As usual we're start with cleaning and attempt to delay EDA until the end.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('../data/train.csv')
print(df.shape)
df.head()

#1.2 million people... no wonder this was such a big file. Realistic considering the industy and what I've worked with
#in the past.

#Recall that premium is this competion's target.

(1200000, 21)


,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
0,0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,...,2.0,17.0,372.0,5.0,2023-12-23 15:21:39.134960,Poor,No,Weekly,House,2869.0
1,1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,...,1.0,12.0,694.0,2.0,2023-06-12 15:21:39.111551,Average,Yes,Monthly,House,1483.0
2,2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,...,1.0,14.0,NaN,3.0,2023-09-30 15:21:39.221386,Good,Yes,Weekly,House,567.0
3,3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,...,1.0,0.0,367.0,1.0,2024-06-12 15:21:39.226954,Poor,Yes,Daily,Apartment,765.0
4,4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,...,0.0,8.0,598.0,4.0,2021-12-01 15:21:39.252145,Poor,Yes,Weekly,House,2022.0


In [3]:
df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace('.', '')

In [4]:
if sum(df.isnull().sum())==0:
    print("Phew, no nulls.")
else:
    print(df.columns[df.isnull().any()])

Index(['age', 'annual_income', 'marital_status', 'number_of_dependents',
       'occupation', 'health_score', 'previous_claims', 'vehicle_age',
       'credit_score', 'insurance_duration', 'customer_feedback'],
      dtype='object')


In [5]:
#After back and fourth with ChatGPT and I, I figured this approach is better. No need to loop.

thresh = .02

null_counts = pd.DataFrame({'null_sum': df.isnull().sum()})
null_counts['ratio'] = round(null_counts['null_sum']/df.shape[0], 3)
null_concerns = null_counts[null_counts['ratio']>thresh].sort_values('ratio', ascending=False)
null_concerns

#So, previous claims has quite a lot of nulls... and even occupation does. Interesting. I wonder if that's th same as unemployed.
#Credit score is always inteesting to see. Dependents are reasonable to ask, as well as feedback. Perhaps some similar issues iwth
#health score as well (ie on the surface I'd assume dubious data doesn't have a health score). No comment on income (in the
#context of nulls).

,null_sum,ratio
previous_claims,364029,0.303
occupation,358075,0.298
credit_score,137882,0.115
number_of_dependents,109672,0.091
customer_feedback,77824,0.065
health_score,74076,0.062
annual_income,44949,0.037


## Initial EDA

Note that this will be my first projec where I hopefully remember to get into the habit of pasting below into a bit of code to later preserve any changes made to the data to a cleaned csv.

In [6]:
i = -1

In [7]:
i += 1 #Eh...
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts(normalize=True)

Now looking at feature 0.


id
0          8.333333e-07
799988     8.333333e-07
800004     8.333333e-07
800003     8.333333e-07
800002     8.333333e-07
               ...     
399999     8.333333e-07
399998     8.333333e-07
399997     8.333333e-07
399996     8.333333e-07
1199999    8.333333e-07
Name: proportion, Length: 1200000, dtype: float64

In [8]:
i += 1
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts(normalize=True)

Now looking at feature 1.


age
53.0    0.022309
61.0    0.022194
39.0    0.022045
64.0    0.022001
57.0    0.021985
43.0    0.021981
62.0    0.021882
46.0    0.021803
33.0    0.021779
47.0    0.021763
44.0    0.021707
34.0    0.021684
58.0    0.021624
31.0    0.021586
56.0    0.021544
32.0    0.021531
54.0    0.021476
38.0    0.021456
36.0    0.021451
35.0    0.021437
22.0    0.021425
59.0    0.021310
37.0    0.021301
51.0    0.021300
55.0    0.021275
49.0    0.021254
45.0    0.021249
20.0    0.021210
21.0    0.021152
50.0    0.021115
40.0    0.021059
48.0    0.021023
26.0    0.020998
24.0    0.020901
30.0    0.020895
29.0    0.020886
19.0    0.020859
42.0    0.020847
60.0    0.020819
52.0    0.020818
18.0    0.020730
28.0    0.020702
63.0    0.020556
25.0    0.020504
27.0    0.020496
41.0    0.020416
23.0    0.019661
Name: proportion, dtype: float64

In [9]:
#Thanks to ChatGPT re. this neat bit right here.

pd.options.display.float_format = '{:,.2f}'.format
df['age'].describe()
# pd.reset_option('display.float_format')

count   1,181,295.00
mean           41.15
std            13.54
min            18.00
25%            30.00
50%            41.00
75%            53.00
max            64.00
Name: age, dtype: float64

Yeah, quite a nice distribution here. Shame it cuts off at 64. Unsurprising that it only starts at 18.

Now, immediately I'd want to band as we assume people of similar ages have similar enough characteristics. Furthermore, from an actuarial/rate perspective we'd do that all the time (for that or whatever other reasns). It definitely makes things simpler. I'm pretty sure even after banding we'd still consider specific ages.

So, starting from 18-22, simplifying all that to the '20' band.... and we'd likely either make the 60 band, consisting of people (by default) 58-62 to be bigger and also have 63 and 64... or a smaller 'old' band of anybody beyond that. Initially I was inclined a bigger final one, but my current desire is to have a smaller older band at our conclusion: It's flexible to account for a theortical change of data (that go beyond 64), retains spirit of my old work where we'd have a senior category, and it's funny (I plan on calling it 'oldies' or 'dinosaurs').

Anyways, essentially normally distributed around 41. Decent standard deviation, but makes sense given stuff (vague).

In [10]:
age_band_names = []
for band in range(20,65, 5):
    age_band_names.append(band)
# print(names)

for feature in age_band_names:
    cond1 = feature - 2
    cond2 = feature + 2
    df[str(feature)] = [1 if (age >= cond1 and age <= cond2) else 0 for age in df['age']]

#ChatGPT recommended something like this, but eh...
# for band in age_band_names[:-1]:  # Handle all except 'dinosaurs'
#     lower, upper = map(int, band.split('-'))
#     df[band] = ((df['age'] >= lower - 2) & (df['age'] <= upper + 2)).astype(int)
    
df['dinosaurs'] = (df['age'] > age_band_names[-1]+2).astype(int)#I'll keep their version. And they were right - why bother
#adding it earlier then have to deal with exception stuff if it's really just this...

In [11]:
pd.reset_option('display.float_format')

i += 1
print(f"Now looking at feature {i}.")

df['gender'] = [1 if gender == 'Male' else 0 for gender in df['gender']]


nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls - deal with them noob.")

df[df.columns[i]].value_counts(normalize=True)

#Nice even split. approximately.



Now looking at feature 2.


gender
1    0.502143
0    0.497858
Name: proportion, dtype: float64

In [12]:
pd.options.display.float_format = '{:,.2f}'.format

# i += 1
print(f"Now looking at feature {i}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls ({round(nulls/df.shape[0], 3)}%) - deal with them noob.")
df[df.columns[i]].describe()

Now looking at feature 2.


count   1,200,000.00
mean            0.50
std             0.50
min             0.00
25%             0.00
50%             1.00
75%             1.00
max             1.00
Name: gender, dtype: float64

In [13]:
#Yeah, I should really keep the filling a seperate cell. Regardless. Let's analyze:
#Unexpectedly skewed to the right, with the median quite lower than the mean... Funny what year the date is based off of
#given inflation and the like. Regardless, the trend holds true.

#I'd expect a strong correlation to health, as by default if they can't even afford healthcare... At the same time, I recall
#the quandry of a certain streamer - if you're broke you get it for free and if you're rich it doesn't matter, but all the
#middl class.... And, evn thte upper tiers I could imagine depending upon their scenarios.... Ie even 150k is not all hat
#much in 2025....

df[df.columns[i]].fillna(df[df.columns[i]].median(), inplace=True)

In [14]:
i += 1
print(f"Now looking at feature {i}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls ({round(nulls/df.shape[0], 3)}%) - deal with them noob.")
df[df.columns[i]].value_counts(normalize=True)

Now looking at feature 3.
Reminder that annual_income has 44949 nulls (0.037%) - deal with them noob.


annual_income
7,073.00     0.00
16,054.00    0.00
14,094.00    0.00
24,897.00    0.00
15,983.00    0.00
             ... 
121,412.00   0.00
54,758.00    0.00
83,944.00    0.00
116,936.00   0.00
40,368.00    0.00
Name: proportion, Length: 88593, dtype: float64

In [15]:
i += 1
print(f"Now looking at feature {i}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls ({round(nulls/df.shape[0], 3)}%) - deal with them noob.")
df[df.columns[i]].value_counts(normalize=True)

Now looking at feature 4.
Reminder that marital_status has 18529 nulls (0.015%) - deal with them noob.


marital_status
Single     0.33
Married    0.33
Divorced   0.33
Name: proportion, dtype: float64

In [16]:
#Even enough distribution. Shame with the Pandas rounding as I'm sure it's not precise. Hmm, maybe I should just do both?
#So, in this case I'd assume single as definitionally they have themselves, so logically it would be the 'baseline'.

df[df.columns[i]].fillna('Single', inplace=True)
df = pd.get_dummies(df, columns=[df.columns[i]], dtype=int, drop_first=True)

In [17]:
#Guess I'll be lazy and not delete this line whenever I get rid of the i += 1 per a dummifying.
print(f"Now looking at feature {i}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls ({round(nulls/df.shape[0], 3)}%) - deal with them noob.")
df[df.columns[i]].value_counts(normalize=True)

Now looking at feature 4.
Reminder that number_of_dependents has 109672 nulls (0.091%) - deal with them noob.


number_of_dependents
3.00   0.20
4.00   0.20
0.00   0.20
2.00   0.20
1.00   0.20
Name: proportion, dtype: float64

In [18]:
resume here next time

NameError: name 'stop' is not defined

In [ ]:
i += 1
print(f"Now looking at feature {i}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls ({round(nulls/df.shape[0], 3)}%) - deal with them noob.")
df[df.columns[i]].value_counts(normalize=True)

In [ ]:
i += 1
print(f"Now looking at feature {i}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls ({round(nulls/df.shape[0], 3)}%) - deal with them noob.")
df[df.columns[i]].value_counts(normalize=True)

In [ ]:
i += 1
print(f"Now looking at feature {i}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls ({round(nulls/df.shape[0], 3)}%) - deal with them noob.")
df[df.columns[i]].value_counts(normalize=True)

In [ ]:
i += 1
print(f"Now looking at feature {i}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls ({round(nulls/df.shape[0], 3)}%) - deal with them noob.")
df[df.columns[i]].value_counts(normalize=True)

In [ ]:
#And finally, the dummifying time. Per my current code of using i do easily go through features I'd rather not potentially
#miss a feature. Ie if I dummify marriag status then the 'new' index is the next feauture. When I then run the code...
#Eh,c ould just delete the i then for the next feature, but whatevr

## Concluding Matters

In [ ]:
stop

In [ ]:
#As I said earlier, prepping the total changed cleanings for modeling.
#Ie kind of like on Git we have the commit before actually pushing.
df = pd.read_csv('../data/train_data.csv')
df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace('.', '')
for col in df.columns:
    try:
        df[col] = round(df[col], 3)
    except:
        pass

age_band_names = []
for band in range(20,65, 5):
    age_band_names.append(band)
# print(names)

for feature in age_band_names:
    cond1 = feature - 2
    cond2 = feature + 2
    df[str(feature)] = [1 if (age >= cond1 and age <= cond2) else 0 for age in df['age']]

#ChatGPT recommended something like this, but eh...
# for band in age_band_names[:-1]:  # Handle all except 'dinosaurs'
#     lower, upper = map(int, band.split('-'))
#     df[band] = ((df['age'] >= lower - 2) & (df['age'] <= upper + 2)).astype(int)
    
df['dinosaurs'] = (df['age'] > age_band_names[-1]+2).astype(int)#I'll keep their version. And they were right - why bother
#adding it earlier then have to deal with exception stuff if it's really just this...

df['gender'] = [1 if gender == 'Male' else 0 for gender in df['gender']]